# GDAL 101

<a target="_blank" href="https://colab.research.google.com/github/mapninja/GDAL-101/blob/main/GDAL-101.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

## Overview of steps:
1. Verification: Verify GDAL installation.
2. Download the data: Placeholder for data download.
3. Check out the data in QGIS: Placeholder for QGIS.
4. OGR for Vector Data: Use GDAL to inspect vector data.
5. GDAL for Raster Data: Use GDAL to inspect raster data.
6. Converting Data: Convert data formats using GDAL.
7. Reprojecting Data: Reproject data using GDAL.
8. Querying Data: Query data using SQL with GDAL.
9. Clipping to create new data: Clip data using GDAL.
10. Previewing Data: Use Folium to preview data.

## Installation

Install necessary packages

In [ ]:
!pip install gdal folium

## Imports

In [2]:
from osgeo import gdal, ogr                     # Import the gdal and ogr modules from the osgeo package
import zipfile
import os
import folium
import geopandas as gpd
import pandas as pd
import json

## Verification:

Verify GDAL installation

In [3]:
gdal.UseExceptions()                            # Enable GDAL to use Python exceptions for error handling
print(gdal.__version__)                         # Print the version of the GDAL library

3.8.4


## Upload and Unzip the data in Colab

This assumes you are working in [Colab](https://colab.research.google.com/). First, you will need to place the `GDAL_101_data.zip` file in the `./contents` folder on Colab. This is the folder that is visible by default, in your Files Panel, in Colab.

In [4]:
# Define the path to the zip file and the extraction directory (in Colab, right-click and select Copy Path to get this location)
zip_file_path = './GDAL_101_data.zip'
extract_to_path = './data'

# Unzip the file
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extract_to_path)

print(f"Data extracted to {os.path.abspath(extract_to_path)}")

Data extracted to /Users/maples/GitHub/GDAL-101/data


## Check out the data in Folium

In [5]:
# Load the shapefile using geopandas
gdf = gpd.read_file(f"{extract_to_path}/zipcodes/zipcodes.shp")

# Convert Timestamp columns to strings for JSON serialization
for col in gdf.columns:
    if pd.api.types.is_datetime64_any_dtype(gdf[col]):
        gdf[col] = gdf[col].astype(str)

# Create a map centered around the data
m = folium.Map(location=[gdf.geometry.centroid.y.mean(), gdf.geometry.centroid.x.mean()], zoom_start=9)

# Add the shapefile layer to the map
folium.GeoJson(gdf).add_to(m)

# Display the map
m

/var/folders/4v/p4krxx750s7bc90tnc6xny0m0000gs/T/ipykernel_89999/396054053.py:10: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  m = folium.Map(location=[gdf.geometry.centroid.y.mean(), gdf.geometry.centroid.x.mean()], zoom_start=9)


## OGR for Vector Data

Inspect vector data

In [7]:
# !ogrinfo ./data/pools/pools.shp                        # Display information about the shapefile 'pools.shp'
# Open the shapefile
shapefile = ogr.Open('./data/pools/pools.shp')

# Get the layer
layer = shapefile.GetLayer()

# Print information about the shapefile
print(f"Layer name: {layer.GetName()}")
print(f"Feature count: {layer.GetFeatureCount()}")



Layer name: pools
Feature count: 51073


In [8]:
# Print the fields in the shapefile
layer_defn = layer.GetLayerDefn()
print("Fields:")
for i in range(layer_defn.GetFieldCount()):
    field_defn = layer_defn.GetFieldDefn(i)
    print(f" - {field_defn.GetName()} ({field_defn.GetTypeName()})")



Fields:
 - POOLS_2013 (Integer)
 - CREATED_BY (String)
 - CREATED_DA (Date)
 - MODIFIED_B (String)
 - MODIFIED_D (Date)
 - SOURCE (String)
 - FEATURE (String)
 - ORIGIN_FEA (String)
 - Shape_Leng (Real)
 - Shape_Area (Real)


In [9]:
# Print the first feature in the shapefile
feature = layer.GetNextFeature()
print("First feature:")
for i in range(layer_defn.GetFieldCount()):
    print(f" - {layer_defn.GetFieldDefn(i).GetName()}: {feature.GetField(i)}")


First feature:
 - POOLS_2013: 1
 - CREATED_BY: AeroMetric
 - CREATED_DA: 2013/10/01
 - MODIFIED_B: None
 - MODIFIED_D: None
 - SOURCE: LiDAR 2012
 - FEATURE: Above ground
 - ORIGIN_FEA: Pools_2013
 - Shape_Leng: 34.4634018685
 - Shape_Area: 94.4298402401


In [10]:
# Open the shapefile
shapefile = ogr.Open(f"{extract_to_path}/zipcodes/zipcodes.shp")

# Get the layer
layer = shapefile.GetLayer()

# Print information about the shapefile
print(f"Layer name: {layer.GetName()}")
print(f"Feature count: {layer.GetFeatureCount()}")



Layer name: zipcodes
Feature count: 80


In [11]:
# Print the fields in the shapefile
layer_defn = layer.GetLayerDefn()
print("Fields:")
for i in range(layer_defn.GetFieldCount()):
    field_defn = layer_defn.GetFieldDefn(i)
    print(f" - {field_defn.GetName()} ({field_defn.GetTypeName()})")

Fields:
 - ZIPCODE (String)
 - NAME (String)
 - CREATED_DA (Date)
 - CREATED_BY (String)
 - MODIFIED_D (Date)
 - MODIFIED_B (String)
 - SHAPE_AREA (Real)
 - SHAPE_LEN (Real)


## GDAL for Raster Data

Inspect raster data

In [15]:
# !gdalinfo ./data/dem10m/dem10m.dem

# Use gdal to get information about the DEM file
dem_info = gdal.Info(f"{extract_to_path}/dem10m/dem10m.dem", format='json')

# Print the information
dem_info


{'description': './data/dem10m/dem10m.dem',
 'driverShortName': 'USGSDEM',
 'driverLongName': 'USGS Optional ASCII DEM (and CDED)',
 'files': ['./data/dem10m/dem10m.dem', './data/dem10m/dem10m.dem.aux.xml'],
 'size': [3600, 3600],
 'coordinateSystem': {'wkt': 'GEOGCRS["NAD83",\n    DATUM["North American Datum 1983",\n        ELLIPSOID["GRS 1980",6378137,298.257222101,\n            LENGTHUNIT["metre",1]]],\n    PRIMEM["Greenwich",0,\n        ANGLEUNIT["degree",0.0174532925199433]],\n    CS[ellipsoidal,2],\n        AXIS["geodetic latitude (Lat)",north,\n            ORDER[1],\n            ANGLEUNIT["degree",0.0174532925199433]],\n        AXIS["geodetic longitude (Lon)",east,\n            ORDER[2],\n            ANGLEUNIT["degree",0.0174532925199433]],\n    ID["EPSG",4269]]',
  'dataAxisToSRSAxisMapping': [2, 1]},
 'geoTransform': [-97.99999940711011,
  0.0002777777777778,
  0.0,
  31.000000503864648,
  0.0,
  -0.0002777777777778],
 'metadata': {'': {'AREA_OR_POINT': 'Point'}},
 'cornerCoor

In [17]:
# Use gdal to get information about the GeoTIFF file
doqq_info = gdal.Info(f"{extract_to_path}/doqq.tif", format='json')

# Print the information
doqq_info

{'description': './data/doqq.tif',
 'driverShortName': 'GTiff',
 'driverLongName': 'GeoTIFF',
 'files': ['./data/doqq.tif'],
 'size': [2394, 2344],
 'coordinateSystem': {'wkt': 'PROJCRS["NAD83 / UTM zone 14N",\n    BASEGEOGCRS["NAD83",\n        DATUM["North American Datum 1983",\n            ELLIPSOID["GRS 1980",6378137,298.257222101,\n                LENGTHUNIT["metre",1]]],\n        PRIMEM["Greenwich",0,\n            ANGLEUNIT["degree",0.0174532925199433]],\n        ID["EPSG",4269]],\n    CONVERSION["UTM zone 14N",\n        METHOD["Transverse Mercator",\n            ID["EPSG",9807]],\n        PARAMETER["Latitude of natural origin",0,\n            ANGLEUNIT["degree",0.0174532925199433],\n            ID["EPSG",8801]],\n        PARAMETER["Longitude of natural origin",-99,\n            ANGLEUNIT["degree",0.0174532925199433],\n            ID["EPSG",8802]],\n        PARAMETER["Scale factor at natural origin",0.9996,\n            SCALEUNIT["unity",1],\n            ID["EPSG",8805]],\n       

## Converting Data

### Exporting original data into new formats

In [ ]:
# List all available formats for ogr2ogr (vector data)
ogr_formats = gdal.VectorTranslateOptions()
print("OGR Formats:")
for i in range(gdal.GetDriverCount()):
    driver = gdal.GetDriver(i)
    if driver and driver.GetMetadataItem(gdal.DCAP_VECTOR):
        print(f" - {driver.ShortName}")

# List all available formats for gdal_translate (raster data)
gdal_formats = gdal.TranslateOptions()
print("\nGDAL Formats:")
for i in range(gdal.GetDriverCount()):
    driver = gdal.GetDriver(i)
    if driver and driver.GetMetadataItem(gdal.DCAP_RASTER):
        print(f" - {driver.ShortName}")


### shapefile → geojson

Convert shapefile to geojson

In [31]:
from osgeo import ogr

# Define the input and output file paths
input_shapefile = './data/zipcodes/zipcodes.shp'
output_geojson = './out/zipcodes.geojson'

# Remove the existing GeoJSON file if it exists
if os.path.exists(output_geojson):
    os.remove(output_geojson)

# Convert shapefile to GeoJSON
shapefile = ogr.Open(input_shapefile)
layer = shapefile.GetLayer()
geojson_driver = ogr.GetDriverByName('GeoJSON')
geojson_ds = geojson_driver.CreateDataSource(output_geojson)
geojson_ds.CopyLayer(layer, 'zipcodes')


<osgeo.ogr.Layer; proxy of <Swig Object of type 'OGRLayerShadow *' at 0x107d921c0> >

In [40]:
import folium
import geopandas as gpd
import os

# Check if the GeoJSON file exists
geojson_path = './out/zipcodes.geojson'
if not os.path.exists(geojson_path):
    raise FileNotFoundError(f"GeoJSON file not found: {geojson_path}")

# Load the GeoJSON file using geopandas
gdf = gpd.read_file(geojson_path)

# Create a map centered around the data
m = folium.Map(location=[gdf.geometry.centroid.y.mean(), gdf.geometry.centroid.x.mean()], zoom_start=9)

# Add the GeoJSON layer to the map
folium.GeoJson(gdf).add_to(m)

# Display the map
m

DriverError: Failed to read GeoJSON data

### GeoTIFF → georeferenced PNG

Convert GeoTIFF to georeferenced PNG

In [ ]:
!gdal_translate -of png ./data/doqq.tif ./out/converted_doqq.png
!gdalinfo ./out/converted_doqq.png

## Reprojecting Data

Reproject data using EPSG codes

In [ ]:
!ogr2ogr -t_srs EPSG:4326 ./out/reprojected_pools.shp ./data/pools/pools.shp
!gdalwarp -t_srs EPSG:4326 ./data/doqq.tif ./out/reprojected_doqq.tif

## Querying Data

Query data using SQL with GDAL

In [43]:
# Display the total count of features in the 'pools' shapefile
!ogrinfo ./data/pools/pools.shp -sql "SELECT COUNT(*) FROM pools"



INFO: Open of `./data/pools/pools.shp'
      using driver `ESRI Shapefile' successful.

Layer name: pools
Geometry: None
Feature Count: 1
Layer SRS WKT:
(unknown)
COUNT_*: Integer (0.0)
OGRFeature(pools):0
  COUNT_* (Integer) = 51073



In [44]:
# Display detailed information about the feature with FID (Feature ID) 1 in the 'pools' shapefile
!ogrinfo -q ./data/pools/pools.shp -sql "SELECT * FROM pools WHERE fid = 1"




Layer name: pools
OGRFeature(pools):1
  POOLS_2013 (Integer) = 2
  CREATED_BY (String) = AeroMetric
  CREATED_DA (Date) = 2013/10/01
  MODIFIED_B (String) = (null)
  MODIFIED_D (Date) = (null)
  SOURCE (String) = LiDAR 2012
  FEATURE (String) = Above ground
  ORIGIN_FEA (String) = Pools_2013
  Shape_Leng (Real) = 44.08991859810
  Shape_Area (Real) = 154.55091064300
  POLYGON Z ((3164378.2166 9997997.058 0,3164378.1782 9997996.3241 0,3164378.063 9997995.5984 0,3164377.8731 9997994.8884 0,3164377.6096 9997994.2024 0,3164377.2759 9997993.5479 0,3164376.8757 9997992.9314 0,3164376.4134 9997992.3605 0,3164375.8937 9997991.8408 0,3164375.3225 9997991.3783 0,3164374.7064 9997990.978 0,3164374.0515 9997990.6447 0,3164373.3655 9997990.3812 0,3164372.6559 9997990.1909 0,3164371.9302 9997990.0761 0,3164371.1962 9997990.0374 0,3164370.4623 9997990.0761 0,3164369.7366 9997990.1909 0,3164369.0269 9997990.3812 0,3164368.3406 9997990.6443 0,3164367.6861 9997990.978 0,3164367.0696 9997991.3783 0,31643

In [45]:
# Display the count of features in the 'pools' shapefile where the 'FEATURE' attribute is 'Above ground'
!ogrinfo -q ./data/pools/pools.shp -sql "SELECT COUNT(*) FROM pools WHERE FEATURE = 'Above ground'"


Layer name: pools
OGRFeature(pools):0
  COUNT_* (Integer) = 21550



### Saving query results to a new file

Create new files based on SQL queries

In [ ]:
!ogr2ogr ./out/austin_zips.shp ./data/zipcodes/zipcodes.shp -sql "SELECT * FROM zipcodes WHERE name = 'Austin'"

In [ ]:
!ogr2ogr ./out/under_10mil.shp ./data/zipcodes/zipcodes.shp -sql "SELECT * FROM zipcodes WHERE SHAPE_AREA <10000000"

In [ ]:
!ogr2ogr -f geojson ./out/78756.geojson ./data/zipcodes/zipcodes.shp -sql "SELECT * FROM zipcodes WHERE ZIPCODE = '78756'"

## Clipping to create new data

Clip data using GDAL

In [ ]:
!ogr2ogr -clipsrc 78756.geojson 78756_pools.shp reprojected_pools.shp
!gdalwarp -cutline 78756.geojson doqq.tif 78756_doqq.tif
!gdal_translate -srcwin 0 0 1000 1000 -of USGSDEM dem10m/dem10m.dem clipped_dem.dem

## Previewing Data

Use Folium to preview data

In [ ]:
import folium
import geopandas as gpd

# Load the geojson file
gdf = gpd.read_file('zipcodes.geojson')

# Create a map centered around the data
m = folium.Map(location=[gdf.geometry.centroid.y.mean(), gdf.geometry.centroid.x.mean()], zoom_start=12)

# Add the geojson layer to the map
folium.GeoJson('zipcodes.geojson').add_to(m)

# Display the map
m